In [172]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage import filters
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, make_path_filter
import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio import plot
import folium
from tqdm import tqdm

In [20]:
# Init Sentinel API
username = 'cryovision' 
password = 'baw.YFV8rfa8nva9mfg' 
api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')

In [21]:
SHAPEFILE = 'pingo_distribution_siberia_ggrosse_bjones/pingo_distribution_siberia_ggrosse_bjones.shp'

In [133]:
# Import Shapefile
pingo = gpd.read_file(SHAPEFILE)
pingo = pingo.to_crs('epsg:4326')
pingo.to_file('pingo.geojson', driver='GeoJSON')


In [ ]:
pingo_number = 19
geojson = read_geojson('pingo.geojson')['features'][pingo_number]
footprint = geojson_to_wkt(geojson)

In [23]:
# Get Images from api

images = api.query(footprint,
                        platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,10),
                     limit=100)
dataframe = api.to_geodataframe(images)
dataframe_sorted = dataframe.sort_values(['ingestiondate','cloudcoverpercentage'], ascending=[False,True])

In [162]:
# # Download Image Data set using Id
path_filter = make_path_filter("*_B0[234]_10m.jp2")
image_id = dataframe_sorted.index[0]
date = dataframe_sorted.generationdate[0].strftime('%d_%m_%Y')
download_data = api.download(image_id,nodefilter=path_filter)

In [25]:
main_path = download_data['node_path'][2:]
image_paths = list(download_data['nodes'].keys())[1:]
b2 = rio.open(main_path+list(download_data['nodes'].keys())[1:][0][1:])
b3 = rio.open(main_path+list(download_data['nodes'].keys())[2:][0][1:])
b4 = rio.open(main_path+list(download_data['nodes'].keys())[3:][0][1:])

In [167]:
os.mkdir(f'images/{image_id}_{date}')
os.mkdir(f'images/{image_id}_{date}/tiles')

in_path = f'images/{image_id}_{date}/'
rgb_filename = 'rgb.tif'
rgb_masked_filename = 'rgb_masked.tif'
out_path = f'images/{image_id}_{date}/tiles/'
output_filename = 'tile_{}-{}.tif'

In [168]:
pingo_proj = pingo.loc[pingo_number:pingo_number].to_crs(b4.crs.data.get('init'))


with rio.open(os.path.join(in_path, rgb_filename),'w',driver='GTiff', width=b4.width, height=b4.height, 
              count=3,crs=b4.crs,transform=b4.transform, dtype=b4.dtypes[0]) as rgb:
    rgb.write(b2.read(1),1) 
    rgb.write(b3.read(1),2) 
    rgb.write(b4.read(1),3) 
    rgb.close()



In [169]:
with rio.open(os.path.join(in_path, rgb_filename)) as src:
    out_image, out_transform = rasterio.mask.mask(src, pingo_proj.geometry.buffer(40000.0),crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
with rasterio.open(os.path.join(in_path, rgb_masked_filename), "w", **out_meta) as dest:
    dest.write(out_image)

In [179]:
import os
from itertools import product
import rasterio as rio
from rasterio import windows

def get_tiles(ds, width=256, height=256):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform


with rio.open(os.path.join(in_path, rgb_masked_filename)) as inds:
    tile_width, tile_height = 256, 256

    meta = inds.meta.copy()
    tiles = list(get_tiles(inds))

    for window, transform in tqdm(tiles):
        meta['transform'] = transform
        meta['width'], meta['height'] = window.width, window.height
        outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
        patch = inds.read(window=window)
        if inds.read(window=window).min()>0 and patch.shape==(3,256,256):
            with rio.open(outpath, 'w', **meta) as outds:
                outds.write(inds.read(window=window))

100%|██████████| 483/483 [00:02<00:00, 229.70it/s]


In [200]:
import folium 
m = folium.Map([pingo.loc[3].LAT, pingo.loc[3].LONG_], zoom_start=6)
folium.GeoJson(data=pingo.geometry).add_to(m)